In [ ]:
!pip install biosimulators-copasi

In [19]:
#define a class which captures output 

from io import StringIO
import sys

class OutputCapturer(list):
    def __init__(self):
        """Implementation: with OutputCapturer() as output: """
        super().__init__()
        self._stdout = None
        self._stringio = None
    
    def __enter__(self):
        self._stdout = sys.stdout
        self._stringio = StringIO()
        sys.stdout = self._stringio
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio
        sys.stdout = self._stdout
        
    def get_lines(self):
        return self.copy()
    
    def get_output(self):
        return '\n'.join(self)


example implementation of ```OutputCapturer()```:



```
def perform_operation(x0, x1):
    return x0*x1 + x1*x0 


with OutputCapturer() as output:
    x0 = 4
    x1 = 6 
    y = perform_operation(x0, x1)
    print(y)

output, y, output.get_output()
```





We attempt to utilize `OutputCapturer()` in place of `StandardOutputErrorCapturer()` in `biosimulators_copasi.core.exec_sed_doc()`. This method will be used as the 'task executer' in the `exec_sedml_docs_in_archive()` method.

In [17]:
from typing import Dict, List, Tuple, Union, Optional  
from biosimulators_utils.combine.exec import exec_sedml_docs_in_archive
from biosimulators_utils.config import get_config, Config  # noqa: F401
from biosimulators_utils.log.data_model import CombineArchiveLog, TaskLog, StandardOutputErrorCapturerLevel, SedDocumentLog  # noqa: F401  
from biosimulators_utils.viz.data_model import VizFormat  # noqa: F401
from biosimulators_utils.report.data_model import ReportFormat, VariableResults, SedDocumentResults, ReportResults  # noqa: F401   
from biosimulators_utils.sedml.data_model import (Task, ModelLanguage, ModelAttributeChange, UniformTimeCourseSimulation,  # noqa: F401
                                                  Variable, Symbol, SedDocument)
from biosimulators_utils.sedml import validation
from biosimulators_utils.sedml.exec import exec_sed_doc as base_exec_sed_doc
from biosimulators_utils.simulator.utils import get_algorithm_substitution_policy
from biosimulators_utils.utils.core import raise_errors_warnings
from biosimulators_utils.warnings import warn, BioSimulatorsWarning
from kisao.data_model import AlgorithmSubstitutionPolicy, ALGORITHM_SUBSTITUTION_POLICY_LEVELS
from biosimulators_copasi.core import exec_sed_task
from biosimulators_copasi.data_model import KISAO_ALGORITHMS_MAP, Units
from biosimulators_copasi.utils import (get_algorithm_id, set_algorithm_parameter_value,
                    get_copasi_model_object_by_sbml_id, get_copasi_model_obj_sbml_ids,
                    fix_copasi_generated_combine_archive as fix_copasi_generated_combine_archive_func)
import COPASI
import lxml
import math
import numpy
import os
import tempfile

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
#with capturer.CaptureOutput(merged=True, relay=False) as captured:

In [24]:
#HERE USE OUTPUT CAPTURER!


def exec_sed_doc(doc: Union[SedDocument, str], working_dir: str, base_out_path: str, 
                 rel_out_path: Optional[str] = None, apply_xml_model_changes: bool = True, 
                 log: Optional[SedDocumentLog] = None, indent: int = 0, 
                 pretty_print_modified_xml_models: bool = False, log_level=None,
                 config: Optional[Config] = None) -> Tuple[ReportResults, SedDocumentLog]:
    """ Execute the tasks specified in a SED document and generate the specified outputs"""
    
    log_level = log_level or OutputCapturer()
    with log_level:
        results, log = base_exec_sed_doc(exec_sed_task, doc, working_dir, base_out_path,
                                         rel_out_path=rel_out_path,
                                         apply_xml_model_changes=apply_xml_model_changes,
                                         log=log,
                                         indent=indent,
                                         pretty_print_modified_xml_models=pretty_print_modified_xml_models,
                                         log_level=log_level,
                                         config=config)
        return {
            'log_level': log_level, 
            'simulation_results': results,
            'log': log
        }

In [ ]:
input_fp = "/content/BIOMD0000000912_sim.sedml"
output_fp = "/content"



7

In [14]:
def exec_sedml_docs_in_combine_archive(archive_filename: str, out_dir: str, config: Optional[Config] = None,
                                       fix_copasi_generated_combine_archive: Optional[bool] = None) -> Tuple[SedDocumentResults, CombineArchiveLog]:
    """ Execute the SED tasks defined in a COMBINE/OMEX archive and save the outputs"""
    
    if fix_copasi_generated_combine_archive is None: # noqa python:S3776
        fix_copasi_generated_combine_archive = os.getenv('FIX_COPASI_GENERATED_COMBINE_ARCHIVE', '0').lower() in ['1', 'true'] # noqa python:S3776

    if fix_copasi_generated_combine_archive: # noqa python:S3776
        temp_archive_file, temp_archive_filename = tempfile.mkstemp()
        os.close(temp_archive_file)
        fix_copasi_generated_combine_archive_func(archive_filename, temp_archive_filename)
        archive_filename = temp_archive_filename

    result = exec_sedml_docs_in_archive(exec_sed_doc, archive_filename, out_dir,
                                        apply_xml_model_changes=True,
                                        config=config)
    if fix_copasi_generated_combine_archive: # noqa python:S3776
        os.remove(temp_archive_filename)

    return result

In [ ]:
try:
        with capturer.CaptureOutput(merged=True, relay=False) as captured:
            exec_sedml_docs_in_archive(
                archive_filename, 
                output_dir, 
                simulator=Simulator(), 
                model_language=model_language, 
                outputs=outputs, 
                sbml_files=sbml_files, 
                validate=validate, 
                **kwargs)

    except Exception as exception:
        # ...
        # Keep the lines that log the captured output
        # ...

    return results, captured.get_captured_text()